In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


In [2]:
kaggle_train_path = "../data/train"
kaggle_test_path = "../data/test"

features_path = "../features"

kaggle_painter_names = os.listdir(kaggle_train_path)

# CNN Model

In [3]:
cnn_height = 255
cnn_width = 255
cnn_batch_size = 32

In [11]:
cnn_train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

cnn_train_generator = cnn_train_datagen.flow_from_directory(
    kaggle_train_path,
    target_size=(cnn_height, cnn_width),
    batch_size=cnn_batch_size,
    class_mode='categorical'
)

cnn_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

cnn_test_generator = cnn_test_datagen.flow_from_directory(
    kaggle_test_path,
    target_size=(cnn_height, cnn_width),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
)

Found 6785 images belonging to 50 classes.
Found 1711 images belonging to 50 classes.


In [19]:
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  # Number of output neurons equal to the number of painters
])

In [20]:
cnn_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

cnn_model.fit(cnn_train_generator, epochs=10)

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy}")

Epoch 1/10
211/211 [==============================] - 238s 1s/step - loss: 3.5128 - accuracy: 0.1228
Epoch 2/10
211/211 [==============================] - 237s 1s/step - loss: 3.1393 - accuracy: 0.1957
Epoch 3/10
211/211 [==============================] - 239s 1s/step - loss: 2.9217 - accuracy: 0.2364
Epoch 4/10
211/211 [==============================] - 235s 1s/step - loss: 2.7695 - accuracy: 0.2717
Epoch 5/10
211/211 [==============================] - 236s 1s/step - loss: 2.6505 - accuracy: 0.2980
Epoch 6/10
211/211 [==============================] - 236s 1s/step - loss: 2.4893 - accuracy: 0.3271
Epoch 7/10
211/211 [==============================] - 235s 1s/step - loss: 2.3830 - accuracy: 0.3614
Epoch 8/10
211/211 [==============================] - 235s 1s/step - loss: 2.2508 - accuracy: 0.3902
Epoch 9/10
211/211 [==============================] - 233s 1s/step - loss: 2.1118 - accuracy: 0.4148
Epoch 10/10
54/54 [==============================] - 25s 453ms/step - loss: 2.3542 - accura

In [21]:
cnn_model.save("../models/cnn_10_epochs_no_validation.h5")

# Optimization of CNN

In [9]:
cnn_model_1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [12]:
optimizer = Adam(learning_rate=0.001)

cnn_model_1.compile(optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],)

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

history = cnn_model_1.fit(cnn_train_generator, epochs=40, callbacks=[early_stopping])

cnn_test_loss_1, cnn_test_accuracy_1 = cnn_model_1.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_1}")

Epoch 1/40
213/213 [==============================] - 237s 1s/step - loss: 3.5118 - accuracy: 0.1293
Epoch 2/40
213/213 [==============================] - 236s 1s/step - loss: 3.1995 - accuracy: 0.1763
Epoch 3/40
213/213 [==============================] - 232s 1s/step - loss: 2.9965 - accuracy: 0.2239
Epoch 4/40
213/213 [==============================] - 235s 1s/step - loss: 2.8387 - accuracy: 0.2604
Epoch 5/40
213/213 [==============================] - 228s 1s/step - loss: 2.7183 - accuracy: 0.2887
Epoch 6/40
213/213 [==============================] - 227s 1s/step - loss: 2.5785 - accuracy: 0.3157
Epoch 7/40
213/213 [==============================] - 227s 1s/step - loss: 2.4585 - accuracy: 0.3431
Epoch 8/40
213/213 [==============================] - 232s 1s/step - loss: 2.3399 - accuracy: 0.3654
Epoch 9/40
213/213 [==============================] - 234s 1s/step - loss: 2.3390 - accuracy: 0.3611
Epoch 10/40
213/213 [==============================] - 231s 1s/step - loss: 2.1907 - accura

In [13]:
cnn_model_1.save("../models/cnn_opt_1.h5")

In [5]:
cnn_train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

cnn_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

cnn_train_generator = cnn_train_datagen.flow_from_directory(
    kaggle_train_path,
    target_size=(cnn_height, cnn_width),
    batch_size=32,
    class_mode="categorical",
    subset="training",  #Use the training subset
)

cnn_validation_generator = cnn_train_datagen.flow_from_directory(
    kaggle_train_path,
    target_size=(cnn_height, cnn_width),
    batch_size=32,
    class_mode="categorical",
    subset="validation",  #Use the validation subset
)

cnn_test_generator = cnn_test_datagen.flow_from_directory(
    kaggle_test_path,
    target_size=(cnn_height, cnn_width),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
)

Found 5451 images belonging to 50 classes.
Found 1334 images belonging to 50 classes.
Found 1711 images belonging to 50 classes.


In [18]:
cnn_model_2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [19]:
optimizer = Adam(learning_rate=0.001)

cnn_model_2.compile(optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],)


early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

train_steps_per_epoch = cnn_train_generator.samples // cnn_train_generator.batch_size
val_steps_per_epoch = cnn_validation_generator.samples // cnn_validation_generator.batch_size

history_2 = cnn_model_2.fit(
    cnn_train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=cnn_validation_generator,
    validation_steps=val_steps_per_epoch,
    epochs=40,
    callbacks=[early_stopping]
)

cnn_test_loss_2, cnn_test_accuracy_2 = cnn_model_2.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_2}")

Epoch 1/40
170/170 [==============================] - 216s 1s/step - loss: 3.6060 - accuracy: 0.1131 - val_loss: 3.3601 - val_accuracy: 0.1380
Epoch 2/40
170/170 [==============================] - 218s 1s/step - loss: 3.3400 - accuracy: 0.1541 - val_loss: 3.1322 - val_accuracy: 0.2020
Epoch 3/40
170/170 [==============================] - 214s 1s/step - loss: 3.1005 - accuracy: 0.1901 - val_loss: 2.9667 - val_accuracy: 0.2424
Epoch 4/40
170/170 [==============================] - 218s 1s/step - loss: 2.9598 - accuracy: 0.2360 - val_loss: 2.8846 - val_accuracy: 0.2546
Epoch 5/40
170/170 [==============================] - 214s 1s/step - loss: 2.8728 - accuracy: 0.2454 - val_loss: 2.7465 - val_accuracy: 0.2790
Epoch 6/40
170/170 [==============================] - 217s 1s/step - loss: 2.7647 - accuracy: 0.2722 - val_loss: 2.6797 - val_accuracy: 0.2919
Epoch 7/40
170/170 [==============================] - 215s 1s/step - loss: 2.6534 - accuracy: 0.2910 - val_loss: 2.6456 - val_accuracy: 0.2973

In [6]:
cnn_model_3 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [9]:
optimizer = Adam(learning_rate=0.01)

cnn_model_3.compile(optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],)


early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

train_steps_per_epoch = cnn_train_generator.samples // cnn_train_generator.batch_size
val_steps_per_epoch = cnn_validation_generator.samples // cnn_validation_generator.batch_size

history_3 = cnn_model_3.fit(
    cnn_train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=cnn_validation_generator,
    validation_steps=val_steps_per_epoch,
    epochs=30,
    callbacks=[early_stopping]
)

cnn_test_loss_3, cnn_test_accuracy_3 = cnn_model_3.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_3}")

Epoch 1/30
170/170 [==============================] - 225s 1s/step - loss: 5.6562 - accuracy: 0.0963 - val_loss: 3.6055 - val_accuracy: 0.1067
Epoch 2/30
170/170 [==============================] - 214s 1s/step - loss: 3.6258 - accuracy: 0.1046 - val_loss: 3.5995 - val_accuracy: 0.1075
Epoch 3/30
170/170 [==============================] - 216s 1s/step - loss: 3.6250 - accuracy: 0.1035 - val_loss: 3.5997 - val_accuracy: 0.0846
Epoch 4/30
170/170 [==============================] - 215s 1s/step - loss: 3.6216 - accuracy: 0.1035 - val_loss: 3.6002 - val_accuracy: 0.1067
Epoch 5/30
170/170 [==============================] - 216s 1s/step - loss: 3.6208 - accuracy: 0.1006 - val_loss: 3.5942 - val_accuracy: 0.1059
Epoch 6/30
170/170 [==============================] - 216s 1s/step - loss: 3.6211 - accuracy: 0.1041 - val_loss: 3.5963 - val_accuracy: 0.1067
Epoch 7/30
170/170 [==============================] - 221s 1s/step - loss: 3.6204 - accuracy: 0.1044 - val_loss: 3.6010 - val_accuracy: 0.1075

KeyboardInterrupt: 

In [13]:
cnn_model_4 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.3),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Dropout(0.4),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [14]:
optimizer = Adam(learning_rate=0.0005)

cnn_model_4.compile(optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],)


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

train_steps_per_epoch = cnn_train_generator.samples // cnn_train_generator.batch_size
val_steps_per_epoch = cnn_validation_generator.samples // cnn_validation_generator.batch_size

history_4 = cnn_model_4.fit(
    cnn_train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=cnn_validation_generator,
    validation_steps=val_steps_per_epoch,
    epochs=40,
    callbacks=[early_stopping]
)

cnn_test_loss_4, cnn_test_accuracy_4 = cnn_model_4.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_4}")

Epoch 1/40
170/170 [==============================] - 243s 1s/step - loss: 3.5395 - accuracy: 0.1319 - val_loss: 3.4530 - val_accuracy: 0.1875
Epoch 2/40
170/170 [==============================] - 234s 1s/step - loss: 3.1874 - accuracy: 0.1995 - val_loss: 3.1792 - val_accuracy: 0.2569
Epoch 3/40
170/170 [==============================] - 241s 1s/step - loss: 2.9510 - accuracy: 0.2353 - val_loss: 3.0678 - val_accuracy: 0.2782
Epoch 4/40
170/170 [==============================] - 236s 1s/step - loss: 2.7857 - accuracy: 0.2755 - val_loss: 2.9761 - val_accuracy: 0.2675
Epoch 5/40
170/170 [==============================] - 230s 1s/step - loss: 2.6484 - accuracy: 0.2967 - val_loss: 2.8841 - val_accuracy: 0.2866
Epoch 6/40
170/170 [==============================] - 233s 1s/step - loss: 2.5138 - accuracy: 0.3331 - val_loss: 2.7309 - val_accuracy: 0.3026
Epoch 7/40
170/170 [==============================] - 240s 1s/step - loss: 2.4038 - accuracy: 0.3567 - val_loss: 2.6790 - val_accuracy: 0.3369

In [27]:
cnn_model_4.save("../models/cnn_opt_4.h5")

In [17]:
cnn_model_5 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='sigmoid'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='tanh'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Dropout(0.4),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [18]:
optimizer = Adam(learning_rate=0.001)

cnn_model_5.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],)


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

train_steps_per_epoch = cnn_train_generator.samples // cnn_train_generator.batch_size
val_steps_per_epoch = cnn_validation_generator.samples // cnn_validation_generator.batch_size

history_5 = cnn_model_5.fit(
    cnn_train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=cnn_validation_generator,
    validation_steps=val_steps_per_epoch,
    epochs=40,
    callbacks=[early_stopping]
)

cnn_test_loss_5, cnn_test_accuracy_5 = cnn_model_5.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_5}")

Epoch 1/40
170/170 [==============================] - 229s 1s/step - loss: 3.8843 - accuracy: 0.0971 - val_loss: 3.7088 - val_accuracy: 0.1067
Epoch 2/40
170/170 [==============================] - 229s 1s/step - loss: 3.6445 - accuracy: 0.1044 - val_loss: 3.5958 - val_accuracy: 0.1059
Epoch 3/40
170/170 [==============================] - 227s 1s/step - loss: 3.6191 - accuracy: 0.1028 - val_loss: 3.5891 - val_accuracy: 0.1067
Epoch 4/40
170/170 [==============================] - 230s 1s/step - loss: 3.6168 - accuracy: 0.1041 - val_loss: 3.5891 - val_accuracy: 0.1082
Epoch 5/40
170/170 [==============================] - 228s 1s/step - loss: 3.6195 - accuracy: 0.1041 - val_loss: 3.5930 - val_accuracy: 0.1075
Epoch 6/40
54/54 [==============================] - 24s 446ms/step - loss: 3.6149 - accuracy: 0.1029
CNN Test Accuracy: 0.10286382585763931


In [26]:
cnn_model_5.save("../models/cnn_opt_5.h5")

In [20]:
cnn_model_6 = Sequential([
    Conv2D(32, (3, 3), activation='sigmoid', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='sigmoid'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='sigmoid'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='sigmoid'),
    Dropout(0.4),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='sigmoid'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [21]:
optimizer = Adam(learning_rate=0.001)

cnn_model_6.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],)


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

train_steps_per_epoch = cnn_train_generator.samples // cnn_train_generator.batch_size
val_steps_per_epoch = cnn_validation_generator.samples // cnn_validation_generator.batch_size

history_6 = cnn_model_6.fit(
    cnn_train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=cnn_validation_generator,
    validation_steps=val_steps_per_epoch,
    epochs=40,
    callbacks=[early_stopping]
)

cnn_test_loss_6, cnn_test_accuracy_6 = cnn_model_6.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_6}")

Epoch 1/40
170/170 [==============================] - 229s 1s/step - loss: 4.2539 - accuracy: 0.0670 - val_loss: 3.6340 - val_accuracy: 0.1082
Epoch 2/40
170/170 [==============================] - 228s 1s/step - loss: 3.9606 - accuracy: 0.0712 - val_loss: 3.6445 - val_accuracy: 0.1059
Epoch 3/40
170/170 [==============================] - 230s 1s/step - loss: 3.9129 - accuracy: 0.0705 - val_loss: 3.6253 - val_accuracy: 0.1067
Epoch 4/40
170/170 [==============================] - 224s 1s/step - loss: 3.8912 - accuracy: 0.0746 - val_loss: 3.6446 - val_accuracy: 0.1052
Epoch 5/40
170/170 [==============================] - 237s 1s/step - loss: 3.8374 - accuracy: 0.0777 - val_loss: 3.6241 - val_accuracy: 0.1067
Epoch 6/40
170/170 [==============================] - 231s 1s/step - loss: 3.8199 - accuracy: 0.0860 - val_loss: 3.6391 - val_accuracy: 0.1052
Epoch 7/40
170/170 [==============================] - 231s 1s/step - loss: 3.8061 - accuracy: 0.0819 - val_loss: 3.6550 - val_accuracy: 0.0846

In [25]:
cnn_model_6.save("../models/cnn_opt_6.h5")

In [22]:
cnn_model_7 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    Dropout(0.5),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [23]:
optimizer = Adam(learning_rate=0.001)

cnn_model_7.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],)


early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

train_steps_per_epoch = cnn_train_generator.samples // cnn_train_generator.batch_size
val_steps_per_epoch = cnn_validation_generator.samples // cnn_validation_generator.batch_size

history_7 = cnn_model_7.fit(
    cnn_train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=cnn_validation_generator,
    validation_steps=val_steps_per_epoch,
    epochs=40,
    callbacks=[early_stopping]
)

cnn_test_loss_7, cnn_test_accuracy_7 = cnn_model_7.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_7}")

Epoch 1/40
170/170 [==============================] - 290s 2s/step - loss: 3.6016 - accuracy: 0.1251 - val_loss: 3.6818 - val_accuracy: 0.0838
Epoch 2/40
170/170 [==============================] - 286s 2s/step - loss: 3.2348 - accuracy: 0.1711 - val_loss: 3.6226 - val_accuracy: 0.1006
Epoch 3/40
170/170 [==============================] - 283s 2s/step - loss: 3.0321 - accuracy: 0.2142 - val_loss: 3.5857 - val_accuracy: 0.1120
Epoch 4/40
170/170 [==============================] - 281s 2s/step - loss: 2.8804 - accuracy: 0.2484 - val_loss: 3.3819 - val_accuracy: 0.1174
Epoch 5/40
170/170 [==============================] - 283s 2s/step - loss: 2.7954 - accuracy: 0.2620 - val_loss: 3.3138 - val_accuracy: 0.1601
Epoch 6/40
170/170 [==============================] - 281s 2s/step - loss: 2.6634 - accuracy: 0.3004 - val_loss: 3.2210 - val_accuracy: 0.1608
Epoch 7/40
170/170 [==============================] - 282s 2s/step - loss: 2.5767 - accuracy: 0.3141 - val_loss: 3.2289 - val_accuracy: 0.1631

In [24]:
cnn_model_7.save("../models/cnn_opt_7.h5")

In [6]:
cnn_model_8 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [8]:
cnn_model_8.add(BatchNormalization())

optimizer = Adam(learning_rate=0.001)

cnn_model_8.compile(optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

history8 = cnn_model_8.fit(
    cnn_train_generator,
    validation_data=cnn_validation_generator,
    epochs=40,
    callbacks=[early_stopping]
)

# Evaluate the model on the test data
cnn_test_loss_8, cnn_test_accuracy_8 = cnn_model_8.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_8}")

Epoch 1/40
171/171 [==============================] - 233s 1s/step - loss: 7.9317 - accuracy: 0.0189 - val_loss: 4.5105 - val_accuracy: 0.0180
Epoch 2/40
171/171 [==============================] - 224s 1s/step - loss: 7.9610 - accuracy: 0.0224 - val_loss: 3.6759 - val_accuracy: 0.0232
Epoch 3/40
171/171 [==============================] - 221s 1s/step - loss: 8.7343 - accuracy: 0.0829 - val_loss: 3.9436 - val_accuracy: 0.0232
Epoch 4/40
171/171 [==============================] - 217s 1s/step - loss: 9.0866 - accuracy: 0.1025 - val_loss: 3.8897 - val_accuracy: 0.0232
Epoch 5/40
54/54 [==============================] - 25s 467ms/step - loss: 3.6695 - accuracy: 0.0228
CNN Test Accuracy: 0.022793687880039215


In [ ]:
cnn_model_9 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(kaggle_painter_names), activation='softmax')  #Number of output neurons equal to the number of painters
])

In [ ]:
cnn_model_9.add(BatchNormalization())

optimizer = Adam(learning_rate=0.001)

cnn_model_9.compile(optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

history8 = cnn_model_9.fit(
    cnn_train_generator,
    validation_data=cnn_validation_generator,
    epochs=40,
    callbacks=[early_stopping]
)

# Evaluate the model on the test data
cnn_test_loss_9, cnn_test_accuracy_9 = cnn_model_8.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy_8}")

# Inception V4

In [22]:
inception_height = 299
inception_width = 299
input_shape = (inception_height, inception_width, 3) #because of RGB

In [23]:
inception_train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

inception_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

inception_train_generator = inception_train_datagen.flow_from_directory(
    kaggle_train_path,
    target_size=(inception_height, inception_width),
    batch_size=32,
    class_mode="categorical",
    subset="training",  #Use the training subset
)

inception_validation_generator = inception_train_datagen.flow_from_directory(
    kaggle_train_path,
    target_size=(inception_height, inception_width),
    batch_size=32,
    class_mode="categorical",
    subset="validation",  #Use the validation subset
)

inception_test_generator = inception_test_datagen.flow_from_directory(
    kaggle_test_path,
    target_size=(inception_height, inception_width),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
)

Found 5411 images belonging to 50 classes.
Found 1324 images belonging to 50 classes.
Found 1711 images belonging to 50 classes.


In [24]:
inception_model = InceptionV3(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape,
)

for layer in inception_model.layers:
    layer.trainable = False

In [25]:
x = GlobalAveragePooling2D()(inception_model.output)

x = Dense(1024, activation="relu")(x)

x = Dense(512, activation="relu")(x)


#dropout, early stop



inception_output = Dense(len(inception_train_generator.class_indices), activation="softmax")(x)

inception = Model(inputs=inception_model.input, outputs=inception_output)

inception.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

In [26]:
train_steps_per_epoch = inception_train_generator.samples // inception_train_generator.batch_size
val_steps_per_epoch = inception_validation_generator.samples // inception_validation_generator.batch_size

inception.fit(
    inception_train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=inception_validation_generator,
    validation_steps=val_steps_per_epoch,
    epochs=10,
)

# Evaluate the model on the test set
inception_test_loss, inception_test_accuracy = inception.evaluate(inception_test_generator)
print(f"Inception Test Accuracy: {inception_test_accuracy}")

Epoch 1/10
169/169 [==============================] - 275s 2s/step - loss: 2.9794 - accuracy: 0.2540 - val_loss: 2.4341 - val_accuracy: 0.3537
Epoch 2/10
169/169 [==============================] - 269s 2s/step - loss: 2.0732 - accuracy: 0.4358 - val_loss: 2.0209 - val_accuracy: 0.4566
Epoch 3/10
169/169 [==============================] - 270s 2s/step - loss: 1.7367 - accuracy: 0.5285 - val_loss: 1.9677 - val_accuracy: 0.4741
Epoch 4/10
169/169 [==============================] - 274s 2s/step - loss: 1.4739 - accuracy: 0.5841 - val_loss: 1.8796 - val_accuracy: 0.4970
Epoch 5/10
169/169 [==============================] - 270s 2s/step - loss: 1.3266 - accuracy: 0.6155 - val_loss: 1.8634 - val_accuracy: 0.5023
Epoch 6/10
169/169 [==============================] - 270s 2s/step - loss: 1.1923 - accuracy: 0.6512 - val_loss: 1.7428 - val_accuracy: 0.5419
Epoch 7/10
169/169 [==============================] - 269s 2s/step - loss: 1.0757 - accuracy: 0.6760 - val_loss: 1.8177 - val_accuracy: 0.5328

In [27]:
inception.save("../models/inceptionv3_10_epochs.h5")